In [4]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
from datetime import datetime, timedelta
import pickle
import cartopy.crs as ccrs
import cartopy.feature as cfeature

plt.rcParams['legend.fontsize'] = 10
plt.rcParams.update({'figure.autolayout': True})

## Loading Data
If a pickled data object exists, load and use it as an instance of `GPS_Stations` class named `Korea_GPS_Stations`.
The loaded pickle file should contain 55 stations.


In [5]:
class GPS_Stations():
    def __init__(self):
        self.data = {}
    def add(self, station_name, date, x, y, z):
        if station_name in self.data:
            # if the station is already in the data dictionary, append a new row of data [date,x,y,z]
            self.data.update( {station_name : np.append( self.data[station_name], np.array([date,x,y,z]).reshape(1,4), axis=0 ) } )
        else: # if first time adding this station, create a key:data pair.
            self.data.update( {station_name : np.array([date,x,y,z]).reshape(1,4)} )

In [8]:
DATA_EXISTS = False
try: 
    Korea_GPS_Stations = pickle.load( open( "Korea_GPS_Stations_20110401_20161231.p", "rb" ) )
    if Korea_GPS_Stations:
        DATA_EXISTS = True
        print("Data object loaded.")
except FileNotFoundError:
    print("Failed to load the pickled data. Generate data with GPSdata.ipynb.")

Data object loaded.


In [9]:
COEFF_EXISTS = False
try: 
    Korea_GPS_CoeffCov = pickle.load( open( "Korea_GPS_Stations_coeff_cov_20110401_20161231.p", "rb" ) )
    if Korea_GPS_CoeffCov:
        COEFF_EXISTS = True
        print("ELTM coefficients and covariances loaded.")
except FileNotFoundError:
    print("Failed to load the pickled data. Generate ELTM coefficients using GPSdata.ipynb.")

ELTM coefficients and covariances loaded.


## Strain rate field by least-square collocation method

### Generate a grid of points

In [24]:
lons = np.linspace(125.0, 131.0, 51)
lats = np.linspace(33.0, 38.5, 51)
long, latg = np.meshgrid(lons, lats)

### Compute $\mathbf{M}$ matrix for each point
A strain rate tensor ($\boldsymbol{\varepsilon}$) at a location is the spatially linear rate of change of velocity at that location. In other words, velocity at a GPS station ($\mathbf{v}$) can be approximated as the sum of a translation vector, $\mathbf{t}$, and the product of strain rate and $\mathbf{X}$, the position vector of the GPS station minus the position vector of a grid point:
$ \mathbf{t} + \boldsymbol{\varepsilon} \mathbf{X} = \mathbf{v}$.

Since we have multiple GPS stations and cannot expect the relationship to hold exactly for all of them, we try to find $\mathbf{t}$ and $\boldsymbol{\varepsilon}$ in the least square sense.
$ \mathbf{t} + \boldsymbol{\varepsilon} \mathbf{X}_{i}^{(i)} = \mathbf{v}^{(i)}$, where $\mathbf{X}^{(i)}$ is the position vector of the $i$-th GPS station minus the position vector of a grid point, and $\mathbf{v}^{(i)}$ is the GPS velocity at the $i$-th station ($i=1,\ldots,n$).

$
\begin{bmatrix}
1 & 0 & X_{1}^{(1)} & X_{2}^{(1)} & 0 & 0 \\
0 & 1 & 0 & 0 & X_{1}^{(1)} & X_{2}^{(1)} \\
1 & 0 & X_{1}^{(2)} & X_{2}^{(2)} & 0 & 0 \\
0 & 1 & 0 & 0 & X_{1}^{(2)} & X_{2}^{(2)} \\
\vdots & \vdots & \vdots & \vdots & \vdots & \vdots \\
1 & 0 & X_{1}^{(n)} & X_{2}^{(n)} & 0 & 0 \\
0 & 1 & 0 & 0 & X_{1}^{(n)} & X_{2}^{(n)}
\end{bmatrix}
\begin{bmatrix}
t_{1} \\
t_{2} \\
\varepsilon_{11} \\
\varepsilon_{12} \\
\varepsilon_{21} \\
\varepsilon_{22}
\end{bmatrix}
=
\begin{bmatrix}
^{1}v_{1} \\
^{1}v_{2} \\
^{2}v_{1} \\
^{2}v_{2} \\
\vdots \\
^{n}v_{1} \\
^{n}v_{2} \\
\end{bmatrix}
$

Putting the above relationship in the form of $\mathbf{M}\mathbf{x} = \mathbf{y}$, we invert it for $\mathbf{x}$ as follows:

$\mathbf{x} = \left( \mathbf{M}^{T}\mathbf{W}\mathbf{M} \right)^{-1} \mathbf{M}^{T}\mathbf{W} \mathbf{y}$, 
where $\mathbf{W}$ is a diagonal matrix of weighting factors.
Among many possible choices for $\mathbf{W}$, we choose the Gaussian function of distance:

$ W = \exp \left( -\frac{d^{2}}{2\alpha^{2}} \right)$
We calculate 

In [37]:
positions = np.vstack([long.ravel(), latg.ravel()])
for position in positions:
    # compute_M
    # invert M for 
from scipy.spatial import distance_matrix
distance_matrix( [[long[0,0],latg[0,0]]], positions )
    
station_loc_dict = pickle.load( open( "Korea_GPS_Stations_locations.p", "rb" ) )

for station, loc in station_loc_dict.items():
    station_loc = np.append(station_loc, [loc])
print(loc, station_loc)

[128.462  37.183] [127.374  36.399 126.462  33.288 127.519  34.454 126.381  34.817 128.744
  35.491 128.457  36.934 128.565  38.251 126.918  37.494 127.27   37.
 127.73   36.488 127.711  37.869 127.155  36.878 126.53   33.514 127.415
  38.164 128.478  35.533 129.056  36.436 128.693  35.236 126.801  36.459
 127.461  36.627 127.061  37.902 125.968  36.125 127.943  35.668 127.787
  36.816 127.709  35.162 129.567  36.078 128.194  37.709 126.686  37.42
 128.171  38.069 126.9    34.675 128.05   35.173 126.974  35.623 127.135
  35.843 128.868  37.771 128.13   36.13  128.574  36.233 126.91   35.178
 126.269  33.117 127.661  36.003 127.396  35.423 127.099  36.186 126.738
  37.746 129.07   35.23  126.494  36.776 128.144  36.379 127.486  34.957
 127.054  37.276 128.802  35.906 130.798  37.518 127.947  37.337 129.416
  35.504 129.413  36.992 127.506  37.454 128.446  36.651 126.516  35.279
 128.462  37.183]
